# Wrangle WikiQA

In [ ]:
import json

import datasets
import pandas as pd

pd.set_option("display.max_colwidth", 1000)

In [ ]:
dataset = datasets.load_dataset("wiki_qa")

In [ ]:
split = "train"
raw_df = dataset[split].to_pandas()
raw_df

In [ ]:
rows = []
for question_id, group in raw_df.groupby(["question_id"], as_index=False):
    questions = group["question"].unique().tolist()
    assert len(questions) == 1
    question = questions[0]
    document_titles = group["document_title"].unique().tolist()
    assert len(document_titles) == 1
    document_title = document_titles[0]
    document_text = " ".join(group["answer"].tolist())
    texts_with_emphasis = []
    for relevance_label, text in zip(group["label"].to_list(), group["answer"].to_list()):
        is_relevant = relevance_label == 1
        if is_relevant:
            texts_with_emphasis.append(text.upper())
        else:
            texts_with_emphasis.append(text)
    document_text_with_emphasis = " ".join(texts_with_emphasis)
    is_relevant = 1 in group["label"].to_list()
    rows.append(
        {
            "query_id": question_id,
            "query_text": question,
            "document_title": document_title,
            "document_text": document_text,
            "document_text_with_emphasis": document_text_with_emphasis,
            "relevant": is_relevant,
        }
    )
df = pd.DataFrame(rows)
df

In [ ]:
data_path = f"wiki_qa-{split}.jsonl"
with open(data_path, "w") as f:
    for record in df.to_dict(orient="records"):
        f.write(json.dumps(record) + "\n")